Calculando métodos e métricas considerando a classificação binária do dataset de creditcardfraud como operações fraudulentas ou não.

In [2]:
import pandas as pd


og_df_pre = pd.read_csv("/mnt/78C91DD876D41E26/GitHub/IA/EP/creditcard.zip")
og_df_norm = pd.read_csv("/mnt/78C91DD876D41E26/GitHub/IA/EP/creditcard_norm.zip")


In [2]:

###
### CROSS VALIDATIONE EXAMPLE ###
###

# from sklearn import model_selection as ms
# from sklearn.model_selection import cross_val_score
# from sklearn import svm

# cols = df_pre.columns.tolist()
# classe = cols.pop(2)

# # Usando apenas 1000 linhas
# a = df_pre.loc[1:1000,cols]
# b = df_pre[classe].iloc[0:1000]

# kf = ms.KFold(10)
# for train_index, test_index in kf.split(X):
#     print("TRAIN:", train_index, "TEST:", test_index)
#     X_train, X_test = X[train_index], X[test_index]
#     y_train, y_test = y[train_index], y[test_index]

# X = a.to_numpy()
# y = b.to_numpy()
# clf = svm.SVC(kernel='linear', C=1, random_state=42)
# scores = cross_val_score(clf, X, y, cv=10, n_jobs=-1)
# print(scores)


## Separando as classes do dataset

In [34]:
from sklearn.model_selection import train_test_split
# Colunas para separar o dataset da classe
cols = og_df_pre.columns.tolist()
classe = cols.pop(2)

num_amostra = 10000
df_pre = og_df_pre.sample(num_amostra, random_state=42)
df_norm = og_df_norm.sample(2*num_amostra, random_state=42)

# todas as linhas
X_pre = df_pre.loc[:,cols]  
y_pre = df_pre[classe].iloc[:]
X_pre_train, X_pre_test, y_pre_train, y_pre_test= train_test_split(X_pre, y_pre, test_size=0.3, random_state=42)


X_norm = df_norm.loc[:,cols]
y_norm = df_norm[classe].iloc[:]
X_norm_train, X_norm_test, y_norm_train, y_norm_test= train_test_split(X_norm, y_norm, test_size=0.3, random_state=42)


## KKN


In [71]:
from sklearn.neighbors import  KNeighborsClassifier

knn_pre = []
knn_norm = []
dist = ["euclidean","manhattan"]
for d in dist:
    for i in range(3,12,2):
        neigh = KNeighborsClassifier(n_neighbors=i, metric=d, n_jobs=-1)
        knn_pre.append({"knn": neigh.fit(X_pre_train,y_pre_train), "dist":d,"k":i})
        knn_norm.append({"knn": neigh.fit(X_norm_train,y_norm_train), "dist":d,"k":i})



## Cross-validation do KNN

In [78]:
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import  KNeighborsClassifier
from sklearn.metrics import classification_report

print("Conjunto antes do Pré-Processamento")
for knn in knn_pre:
    y_pred = knn["knn"].predict(X_pre_test)
    cr = classification_report(y_pre_test,y_pred,target_names=["0","1"],zero_division=0)
    print(cr)
    # scores = cross_val_score(knn["knn"],X_pre_test,y=y_pre_test,cv=10,n_jobs=-1)
    # print(f"K: {i} | Distância: {d} | Acurácia {scores.mean():.6f} com D.P. de {scores.std():.6f}")



Conjunto antes do Pré-Processamento
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2993
           1       0.00      0.00      0.00         7

    accuracy                           1.00      3000
   macro avg       0.50      0.50      0.50      3000
weighted avg       1.00      1.00      1.00      3000

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2993
           1       0.00      0.00      0.00         7

    accuracy                           1.00      3000
   macro avg       0.50      0.50      0.50      3000
weighted avg       1.00      1.00      1.00      3000

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2993
           1       0.00      0.00      0.00         7

    accuracy                           1.00      3000
   macro avg       0.50      0.50      0.50      3000
weighted avg       1.00      1.00     

## MLP

In [59]:
from sklearn.neural_network import MLPClassifier
range_mlp = range(1,3)

mlp_pre = []
mlp_norm = []
for n_layer in range_mlp:
    clf = MLPClassifier(solver='adam',alpha=1e-5,hidden_layer_sizes=(n_layer,), random_state=42)
    mlp_pre.append({"mlp": clf.fit(X_pre_train,y_pre_train), "hidden":n_layer})
    mlp_norm.append({"mlp": clf.fit(X_norm_train,y_norm_train), "hidden":n_layer})


In [61]:
for mlp in mlp_pre:
    print(mlp["mlp"].predict(X_pre_test))
#     score = mlp["mlp"].score(X_pre_test,y_pre_test)
#     print(f"Hidden layers: {mlp['hidden']} | Acurácia de {score.mean():.6f} com DP de {score.std():.6f}")
# print("\nConjunto Normalizado")
# for mlp in mlp_norm:
#     score = mlp["mlp"].score(X_norm_test,y_norm_test)
#     print(f"Hidden layers: {mlp['hidden']} | Acurácia de {score.mean():.6f} com DP de {score.std():.6f}")

[1. 1. 1. ... 1. 1. 1.]
[0. 0. 0. ... 0. 0. 0.]


## Cross-validation do MLP

In [68]:
# from sklearn.model_selection import cross_val_score
# from sklearn.metrics import classification_report

# print("Conjunto Pré-Processamento")
# for mlp in mlp_pre:
#     y_pred = mlp["mlp"].predict(X_pre_test)
#     print(y_pred)
#     print(y_pre_test.values)
#     print(f"Hidden layers: {mlp['hidden']} | Acurácia de {score.mean():.6f} com DP de {score.std():.6f}")
# # print("\nConjunto Normalizado")
# # for mlp in mlp_norm:
# #     score = mlp["mlp"].score(X_norm_test,y_norm_test)
# #     print(f"Hidden layers: {mlp['hidden']} | Acurácia de {score.mean():.6f} com DP de {score.std():.6f}")

Conjunto Pré-Processamento
[1. 1. 1. ... 1. 1. 1.]
[0 0 0 ... 0 0 0]
Hidden layers: 1 | Acurácia de 0.493833 com DP de 0.000000
[0. 0. 0. ... 0. 0. 0.]
[0 0 0 ... 0 0 0]
Hidden layers: 2 | Acurácia de 0.493833 com DP de 0.000000
